https://spacy.io/universe/project/medspacy

ne semble fonctionner que pour l'anglais

In [27]:
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent
import spacy
# Load medspacy model
#nlp = medspacy.load()
#print(nlp.pipe_names)

nlp = spacy.load("en_core_web_sm", disable={"parser","ner"})
nlp = medspacy.load(nlp)

text = """
Past Medical History:
1. Atrial fibrillation
2. Type II Diabetes Mellitus

Assessment and Plan:
There is no evidence of pneumonia. Continue warfarin for Afib. Follow up for management of type 2 DM.

question: Of the following statements about the hepatitis B virus, only one is false,
        answers: 
            It is an RNA virus
            It belongs to the Hepadnaviridae family
           It has a lipoprotective envelope
            It has a capsid
           It can be transmitted by blood


"""

# Add rules for target concept extraction
target_matcher = nlp.get_pipe("medspacy_target_matcher")
target_rules = [
    TargetRule("atrial fibrillation", "PROBLEM"),
    TargetRule("atrial fibrillation", "PROBLEM", pattern=[{"LOWER": "afib"}]),
    TargetRule("pneumonia", "PROBLEM"),
    TargetRule("Type II Diabetes Mellitus", "PROBLEM", 
              pattern=[
                  {"LOWER": "type"},
                  {"LOWER": {"IN": ["2", "ii", "two"]}},
                  {"LOWER": {"IN": ["dm", "diabetes"]}},
                  {"LOWER": "mellitus", "OP": "?"}
              ]),
    TargetRule("warfarin", "MEDICATION")
]
target_matcher.add(target_rules)

doc = nlp(text)
visualize_ent(doc)

for ent in doc.ents:
    print(ent, ent._.is_negated, ent._.is_family, ent._.is_historical)

Atrial fibrillation False False False
Type II Diabetes Mellitus False False False
pneumonia True False False
warfarin False False False
Afib False False False
type 2 DM False False False


In [22]:
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent

#nlp = medspacy.load()
nlp = spacy.load("fr_core_news_sm", disable={"parser","ner"})
nlp = medspacy.load(nlp)
print(nlp.pipe_names)

nlp.get_pipe('medspacy_target_matcher').add([TargetRule('stroke', 'CONDITION'), TargetRule('diabétique', 'CONDITION'), TargetRule('pna', 'CONDITION')])
doc = nlp("Le patient a un antécédent d'accident vasculaire cérébral. Sa mother a été diagnostiquée diabétique. Pas de preuve de l'existence de l'angine de poitrine.")

for ent in doc.ents:
    print(ent, ent._.is_negated, ent._.is_family, ent._.is_historical)
#medspacy.visualization.visualize_ent(doc)
visualize_ent(doc)

['tok2vec', 'morphologizer', 'attribute_ruler', 'lemmatizer', 'medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']
diabétique False True False


https://spacy.io/universe/project/edsnlp

Pour l'annotation en français avec spacy et des regex, me demande comment on peut s'en servir pour le moment ?

In [30]:
import spacy

nlp = spacy.blank("fr")

terms = dict(
    covid=["covid", "coronavirus"],
)

# Sentencizer component, needed for negation detection
nlp.add_pipe("eds.sentences")
# Matcher component
nlp.add_pipe("eds.matcher", config=dict(terms=terms))
# Negation detection
nlp.add_pipe("eds.negation")

# Process your text in one call !
doc = nlp("Le patient est atteint de covid")

doc.ents
# Out: (covid,)

#doc.ents[0]._.negation
# Out: False



Adding terms into the pipeline: 100%|████████████| 7/7 [00:00<00:00, 508.12it/s]


(patient, covid)

In [6]:
import spacy

# Declare the pipeline
nlp = spacy.blank("eds")

# General-purpose components
nlp.add_pipe("eds.normalizer")
nlp.add_pipe("eds.sentences")

# Entity extraction pipelines
nlp.add_pipe("eds.drugs", config=dict(term_matcher="simstring"))
nlp.add_pipe("eds.covid")
nlp.add_pipe("eds.dates")
nlp.add_pipe("eds.measurements")
nlp.add_pipe("eds.charlson")
nlp.add_pipe("eds.SOFA")
nlp.add_pipe("eds.emergency.priority")
nlp.add_pipe("eds.adicap")

# RegEx and terms matcher
nlp.add_pipe(
    "eds.matcher",
    config=dict(
        regex=dict(custom=r"asthmatique|difficult[ée]s?\srespiratoires?"),
        attr="NORM",
    ),
)

# Qualifier pipelines
nlp.add_pipe("eds.negation")
nlp.add_pipe("eds.family")
nlp.add_pipe("eds.hypothesis")
nlp.add_pipe("eds.reported_speech")

# Define the note text
text = """
Motif :
Le patient est admis le 29 août pour des difficultés respiratoires.

Antécédents familiaux :
Le père du patient n'est pas asthmatique.

HISTOIRE DE LA MALADIE
Le patient dit avoir de la toux depuis trois jours. Elle a empiré jusqu'à nécessiter un passage aux urgences.
A noter deux petits kystes bénins de 1 et 2cm biopsiés en 2005.

Priorité: 2 (établie par l'IAO à l'entrée)

adicaps ABCD0A12 et ABCD0A13

Conclusion
Possible infection au coronavirus. Prescription de paracétomol pour la fièvre.
"""

# Apply the pipeline
doc = nlp(text)

# Explore matched elements
doc.ents

100%|█████████████████████████████████████| 1968/1968 [00:00<00:00, 2707.72it/s]
Adding terms into the pipeline: 0it [00:00, ?it/s]
Adding terms into the pipeline: 100%|███████████| 1/1 [00:00<00:00, 1189.20it/s]


Adding terms into the pipeline: 100%|███████████| 1/1 [00:00<00:00, 4821.04it/s]
Adding terms into the pipeline: 0it [00:00, ?it/s]
Adding terms into the pipeline: 100%|████████████| 3/3 [00:00<00:00, 158.08it/s]


(difficultés respiratoires,
 asthmatique,
 Priorité: 2,
 ABCD0A12,
 ABCD0A13,
 infection au coronavirus,
 paracétomol)

In [11]:
           
import spacy

# Declare the pipeline
nlp = spacy.blank("eds")

# General-purpose components
nlp.add_pipe("eds.normalizer")
nlp.add_pipe("eds.sentences")

# Entity extraction pipelines
nlp.add_pipe("eds.drugs", config=dict(term_matcher="simstring"))
nlp.add_pipe("eds.covid")
nlp.add_pipe("eds.dates")
nlp.add_pipe("eds.measurements")
nlp.add_pipe("eds.charlson")
nlp.add_pipe("eds.SOFA")
nlp.add_pipe("eds.emergency.priority")
nlp.add_pipe("eds.adicap")

# RegEx and terms matcher
nlp.add_pipe(
    "eds.matcher",
    config=dict(
        regex=dict(custom=r"enzyme|lipides alimentaires|lipase"),
        attr="NORM",
    ),
)

# Qualifier pipelines
nlp.add_pipe("eds.negation")
nlp.add_pipe("eds.family")
nlp.add_pipe("eds.hypothesis")
nlp.add_pipe("eds.reported_speech")

# Define the note text
text ="""
question: Parmi les propositions suivantes, une seule est fausse. Indiquer laquelle? Les enzymes suivantes interviennent dans la digestion des lipides alimentaires:,
        answers: {
            a: "Lipase gastrique",
            b: "Lipase pancréatique",
            c: "Lipoprotéine lipase",
            d: "Phospholipase A2",
            e: "Cholestérol estérase"
"""  

# Apply the pipeline
doc = nlp(text)

# Explore matched elements
doc.ents

for ent in doc.ents:
    print(ent.)

100%|█████████████████████████████████████| 1968/1968 [00:00<00:00, 3154.87it/s]
Adding terms into the pipeline: 0it [00:00, ?it/s]
Adding terms into the pipeline: 100%|███████████| 1/1 [00:00<00:00, 1457.37it/s]


Adding terms into the pipeline: 100%|███████████| 1/1 [00:00<00:00, 5793.24it/s]
Adding terms into the pipeline: 0it [00:00, ?it/s]
Adding terms into the pipeline: 100%|████████████| 3/3 [00:00<00:00, 118.77it/s]

custom
custom
custom
custom
custom
custom
